In [9]:
import numpy as np
#from bitstream import BitStream
from numpy import array


In [2]:
class ArithmeticEncoder(object):
    def __init__(self, bitlen):
        self.bit_prec = bitlen   #bit precision 
        self.max_range = 1 << self.bit_prec  #max range based on bit precision 2^bit_prec
        self.mask = self.max_range - 1  #max range index starting at 0
        self.renorm= self.max_range >> 1  #renormalization threshold
        self.second_mask = self.max_range >> 1
        self.low = 0  #initial low
        self.high = self.mask  #initial high
        self.s = 0  

    def update(self, sym, c):
        low = self.low   
        high = self.high
        range = high - low + 1
#         print(sym)
#         print(sym.shape)
        sym = int(sym)
        total = (c[-1])  #cumulative propabilities
        symlow = (c[sym])  
        symhigh = (c[sym+1] )
        

        newlow = low + symlow*range // total  #low in arithmetic integer
        newhigh = low + symhigh*range // total -1 #high in arithemtic integer
        self.low = int(newlow)
        self.high = int(newhigh)
        range = self.high - self.low 
#         print("sym:", sym)
#         print("symlow:", self.low)
#         print("symhigh:", self.high)
#         print("range:", range)
        
        #renormalization
        while((self.low ^ self.high) & self.renorm) == 0:
            self.low = (self.low << 1) 
            range = range << 1 | 1
            self.high = self.low + range
            self.s = self.s + 1          
#         print("renorm low:",self.low)
#         print("renorm range:",range)
#         print("renorm high:", self.high)
   
    def write(self, c, sym):
        self.update(c,sym)
        return [self.low, self.s]
        
    def finish(self):
        self.output.write(1)


# In[25]:


In [3]:
class ArithmeticDecoder(ArithmeticEncoder):
    def __init__(self,statesize,bitin,c):
        self.input = bitin
        self.bitstream = ([int(d) for d in str(self.input)])
        self.code = 0
        self.max_range = 1 << statesize
        self.renorm= self.max_range >> 1
        self.mask = self.max_range - 1
        self.stream = self.input[0:statesize]
        self.low = int(self.stream,2)
        self.t = statesize
        self.thresh = []
        self.total = (c[-1])
        self.cum = [c[i]/self.total for i in range(len(c))]
        self.thresh = [round((self.cum[i])*self.mask) for i in range(len(self.cum))]
        
    def decode(self, c):
#         print("low:",self.low
        c = [c[i]/self.total for i in range(len(c))]
#         print("cum::", c)
        for i in range(len(self.thresh)-1):
            if((self.low < self.thresh[i+1]) & (self.low >= self.thresh[i])):
                sym = i
#                 print("encoded value:",self.low)
#                 print("threshold:", self.thresh)
#                 print("decoded value:", sym)
                rangenew = int(self.thresh[i+1] - self.thresh[i])
#                 print("range:", rangenew)
#                 print("low:", self.thresh[i])
                while (rangenew < self.renorm):
                    rangenew = rangenew << 1 
                    self.low = self.low << 1
                    self.thresh[i] = int(self.thresh[i]) << 1
#                     print("renorm val;",self.low)
#                     print("renorm range;",rangenew)
                    self.thresh[i+1] = rangenew + self.thresh[i]
#                     print("renorm low:" ,self.thresh[i])
                self.thresh = [(c[j]*(self.thresh[i+1]-self.thresh[i]))+self.thresh[i] for j in range(len(self.cum))]
                break
        return sym


In [4]:
#test bit stream
bitprecision = 64
alphabet_size = 4
#fake input stream -- use real data
sym = np.array([1, 2, 2 ,0 ,3 ,3 ,1 ,2 ,2 ,0, 3, 3 ,1 ,2 ,2, 0, 3, 3 ,1 ,2, 2 ,2 ,3, 3 ,1 ,2 ,2 ,0, 3 ,3 ,1 ,2, 2 ,0 ,3 ,3 ,1,
 2 ,2 ,0 ,3,3 ,1 ,2 ,2, 0 ,3 ,3, 1, 2 ,2, 0, 3, 3, 1, 2, 2, 0 ,3 ,3, 1, 3 ,2 ,0])
#make an array of cumulative probabilities
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.uint64)
c[1:] = np.cumsum(prob*10000000 + 1)  


    
# print("symbol stream input:", in_stream)
print("num stream:", sym)
print("probability:", prob)
print("cumulative prob:", c)
enc = ArithmeticEncoder(bitprecision)
new_l = []
for i in range((sym.shape[0])):
    if (i%16 == 0 and i != 0):
        enc = None
        enc = ArithmeticEncoder(bitprecision)
        new_l.append(new)
    new = enc.write(sym[i], c.tolist())
new_l.append(new)

symdec = []
new = new_l[0]
count = 1
for i in range(len(new_l)):
    new = new_l[i]
    count = count + 1
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream
    #START DECODER
    dec = None
    dec = ArithmeticDecoder(bitprecision, bitstream, c.tolist())
    for i in range(16):
        symdec.append(dec.decode(c.tolist()))   


print("decoded num stream:", symdec)
print(len(symdec))



num stream: [1 2 2 0 3 3 1 2 2 0 3 3 1 2 2 0 3 3 1 2 2 2 3 3 1 2 2 0 3 3 1 2 2 0 3 3 1
 2 2 0 3 3 1 2 2 0 3 3 1 2 2 0 3 3 1 2 2 0 3 3 1 3 2 0]
probability: [0.25 0.25 0.25 0.25]
cumulative prob: [       0  2500001  5000002  7500003 10000004]
decoded num stream: [1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 2, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 3, 2, 0]
64


In [4]:
#test bit stream
bitprecision = 64
alphabet_size = 4
#fake input stream -- use real data
sym = np.array([1, 2, 2 ,0 ,3 ,3 ,1 ,2 ,2 ,0, 3, 3 ,1 ,2 ,2, 0, 3, 3 ,1 ,2, 2 ,2 ,3, 3 ,1 ,2 ,2 ,0, 3 ,3 ,1 ,2, 2 ,0 ,3 ,3 ,1,
 2 ,2 ,0 ,3,3 ,1 ,2 ,2, 0 ,3 ,3, 1, 2 ,2, 0, 3, 3, 1, 2, 2, 0 ,3 ,3, 1, 3 ,2 ,0])

In [11]:
c_m = np.load('c_m.npy')

In [28]:
enc = ArithmeticEncoder(bitprecision)
new_l = []
for i in range((sym.shape[0])):
    if (i%16 == 0 and i != 0):
        enc = None
        enc = ArithmeticEncoder(bitprecision)
        new_l.append(new)
    c = c_m[i, :]
    new = enc.write(sym[i], c.tolist())
new_l.append(new)

In [32]:
dec = None
c = c_m[0, :]
symdec = []
new = new_l[0]
count = 0
for i in range(len(new_l)):
    c = c_m[i, :]
    dec = ArithmeticDecoder(bitprecision, bitstream, c.tolist())
    new = new_l[i]
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream
    #START DECODER
    for j in range(16):
        c = c_m[count, :]
        symdec.append(dec.decode(c.tolist()))   
        count = count + 1

print("decoded num stream:", symdec)
print(len(symdec))


decoded num stream: [3, 2, 2, 3, 1, 2, 0, 1, 0, 0, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 0, 1, 0, 3, 2, 1, 3, 2, 0, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 2, 3, 0, 0, 3, 3, 2, 3, 0, 2, 0, 2, 2, 0, 1, 2, 3, 1, 3, 0, 1, 3, 0, 1, 0]
64


In [33]:
sym

array([1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 2,
       3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2,
       2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3, 3, 1, 3, 2, 0])

In [5]:
int(sym.shape[0]+1 / 16)

64

In [8]:
(sym.shape[0]+1) /16
    

4.0625

In [18]:
sym = np.array([1, 2, 2 ,0 ,3 ,3 ,1 ,2 ,2 ,0, 3, 3 ,1 ,2 ,2, 0, 3, 3 ,1 ,2, 2 ,0 ,3, 3 ,1 ,2 ,2 ,0, 3 ,3 ,1 ,2, 2 ,0 ,3 ,3 ,1,
 2 ,2 ,0 ,3,3 ,1 ,2 ,2, 0 ,3 ,3, 1, 2 ,2, 0, 3, 3, 1, 2, 2, 0 ,3 ,3, 1, 2 ,2 ,0])